In [1]:
import torch
import numpy as np
import xgboost as xgb

from sklearn.metrics import mean_squared_error

from XGBoostTools import *

In [2]:
td, tdtest = LoadDibosonData(int(1e7))


Reading file .../data3/WZ_new_project/xgboost/../h5/Ideal_Reweighted_Latent_Data_Coefficients//trainingSampleReweightedLarge_Latent.h5
Data Type = torch.float32. Global variable torch.set_default_dtype set to torch.float32.

Reading file .../data3/WZ_new_project/xgboost/../h5/Ideal_Reweighted_Latent_Data_Coefficients//testingSampleReweightedLarge_Latent.h5
Data Type = torch.float32. Global variable torch.set_default_dtype set to torch.float32.


### default xgboost classifier

In [47]:
dtrain, dtest, smweights, bsmweights = ConstructDMatrixData(td, tdtest, gw=0.01)

In [32]:
regressor = xgb.train({'tree_method': 'hist', 'seed': 1994},
                      dtrain=dtrain,
                      num_boost_round=10,)

y_pred = regressor.predict(dtest)

NeymanPearsonTestXGBoost(y_pred, smweights, bsmweights, 3000)

tensor([0.0523, 0.0034], device='cuda:0')

### customised loss: quadratic

In [33]:
def gradient(predt: np.ndarray, dtrain: np.ndarray) -> np.ndarray:
    y = dtrain.get_label()
    w = dtrain.get_weight()
    return 2.*w*(predt - y)

def hessian(predt: np.ndarray, dtrain: np.ndarray) -> np.ndarray:
    y = dtrain.get_label()
    w = dtrain.get_weight()
    return 2.*w

def my_quadratic(predt: np.ndarray, dtrain: np.ndarray)  -> np.ndarray:
    grad = gradient(predt, dtrain)
    hess = hessian(predt, dtrain)
    return grad, hess

In [47]:
dtrain, dtest, smweights, bsmweights = ConstructDMatrixData(td, tdtest, gw=0.01)

In [34]:
regressor_custom = xgb.train({'tree_method': 'hist', 'seed': 1994},
                      dtrain=dtrain,
                      num_boost_round=10,
                      obj=my_quadratic)

y_pred = regressor_custom.predict(dtest)

NeymanPearsonTestXGBoost(y_pred, smweights, bsmweights, 3000)

tensor([0.0522, 0.0031], device='cuda:0')

### learning a and b separately

In [3]:
td, tdtest = LoadDibosonData(int(1e7))


Reading file .../data3/WZ_new_project/xgboost/../h5/Ideal_Reweighted_Latent_Data_Coefficients//trainingSampleReweightedLarge_Latent.h5
Data Type = torch.float32. Global variable torch.set_default_dtype set to torch.float32.

Reading file .../data3/WZ_new_project/xgboost/../h5/Ideal_Reweighted_Latent_Data_Coefficients//testingSampleReweightedLarge_Latent.h5
Data Type = torch.float32. Global variable torch.set_default_dtype set to torch.float32.


In [4]:
def ConstructDTrain2D(td, log=False):    
    data      = td.Data.numpy()
    
    if log:
        data[:, 0] = np.log(data[:, 0])
        data[:, 4] = np.log(data[:, 4])
    
    a, b      = td.ReweightCoeffs[:, 1], td.ReweightCoeffs[:, 3]
    
    stats     = {'a_mean': a.mean(), 'a_std': a.std(), 
                'b_mean': b.mean(), 'b_std': b.std()}
    
    a, b      = (a - a.mean())/a.std(), (b - b.mean())/b.std()
        
    dtrain    = [xgb.DMatrix(data, label=a), xgb.DMatrix(data, label=b)]
            
    return dtrain, stats, data

def ConstructDTest(tdtest, gphi=0., gw=0., log=False):
    rwval      = torch.Tensor([gphi, gw])
    wilson     = torch.cat([rwval, rwval**2, rwval.prod().reshape(1)]).reshape(-1, 1)

    datatest   = tdtest.Data.numpy()
    
    if log:
        datatest[:, 0] = np.log(datatest[:, 0])
        datatest[:, 4] = np.log(datatest[:, 4])
    
    dtest      = xgb.DMatrix(datatest)
    
    smweights  = tdtest.Weights
    bsmweights = tdtest.ReweightCoeffs.mm(wilson).flatten()
    
    return dtest, smweights, bsmweights, datatest

In [5]:
dtrain, stats, data = ConstructDTrain2D(td)

regressor_a = xgb.train({'tree_method': 'hist'},
                      dtrain=dtrain[0],
                      num_boost_round=10)

regressor_b = xgb.train({'tree_method': 'hist'},
                      dtrain=dtrain[1],
                      num_boost_round=10)

In [7]:
dtest, smweights, bsmweights, datatest =  ConstructDTest(tdtest, gw=0.01)

a_pred = regressor_a.predict(dtest)*stats['a_std'].numpy() + stats['a_mean'].numpy()
b_pred = regressor_b.predict(dtest)*stats['b_std'].numpy() + stats['b_mean'].numpy()
reweights_pred = smweights + a_pred*0.02 + b_pred*(0.02**2)
rho_pred       = reweights_pred/smweights
y_pred         = rho_pred/(1. + rho_pred)
y_pred

tensor([0.5062, 0.5052, 0.5116,  ..., 0.5053, 0.4951, 0.4995])

In [8]:
NeymanPearsonTestXGBoost(y_pred, smweights, bsmweights, 3000)

tensor([0.0576, 0.0038], device='cuda:0')

### Parameter tunining

In [9]:
dtrain, stats, data = ConstructDTrain2D(td)

In [ ]:
regressor_a=xgb.XGBRegressor(eval_metric='rmsle', tree_method='gpu_hist')

regressor_b=xgb.XGBRegressor(eval_metric='rmsle', tree_method='gpu_hist')

#=========================================================================
# exhaustively search for the optimal hyperparameters
#=========================================================================
from sklearn.model_selection import GridSearchCV
# set up our search grid
param_grid = {"n_estimators": [10, 20, 50, 100],
             "max_depth": [5, 10, 20, 50]}

# try out every combination of the above values
search_a = GridSearchCV(regressor_a, param_grid, cv=5).fit(data, dtrain[0].get_label())

search_b = GridSearchCV(regressor_b, param_grid, cv=5).fit(data, dtrain[1].get_label())

print("The best hyperparameters are ", search.best_params_)

In [ ]:
regressor_a =xgb.XGBRegressor(n_estimators  = search_a.best_params_["n_estimators"],
                           max_depth     = search_a.best_params_["max_depth"], tree_method='gpu_hist')

regressor_a.fit(data, dtrain[0].get_label())

regressor_b = xgb.XGBRegressor(n_estimators  = search_b.best_params_["n_estimators"],
                           max_depth     = search_b.best_params_["max_depth"],  tree_method='gpu_hist')

regressor_b.fit(data, dtrain[1].get_label())

In [ ]:
dtest, smweights, bsmweights, datatest =  ConstructDTest(tdtest, gw=0.01)

a_pred = regressor_a.predict(datatest)*stats['a_std'].numpy() + stats['a_mean'].numpy()
b_pred = regressor_b.predict(datatest)*stats['b_std'].numpy() + stats['b_mean'].numpy()
reweights_pred = smweights + a_pred*0.02 + b_pred*(0.02**2)
rho_pred       = reweights_pred/smweights
y_pred         = rho_pred/(1. + rho_pred)
print(y_pred)

NeymanPearsonTestXGBoost(y_pred, smweights, bsmweights, 3000)

In [101]:
#a_pred  = y_pred[:, 0]*stats['a_std'].numpy() + stats['a_mean'].numpy()
#b_pred = y_pred[:, 1]*stats['b_std'].numpy() + stats['b_mean'].numpy()

In [78]:
# numpy version with gradient and hessian
#Manual calculate the gradient and hessian then implement the custom metric
def gradient(predt: np.ndarray, dtrain: np.ndarray) -> np.ndarray:
    y = dtrain.get_label()
    return (np.log(predt)-np.log1p(y))/(predt+1)

def hessian(predt: np.ndarray, dtrain: np.ndarray) -> np.ndarray:
    y = dtrain.get_label()
    return (-np.log1p(predt)+np.log1p(y)+1)/np.power(predt+1, 2)
    
#return the grad and hessian
def squared_log(predt: np.ndarray, dtrain: np.ndarray) -> np.ndarray:
    predt[predt < -1] = -1+1e-6
    grad = gradient(predt, dtrain)
    hess = hessian(predt, dtrain)
    return grad, hess


def gradient(predt: np.ndarray, dtrain: np.ndarray) -> np.ndarray:
    y = dtrain.get_label()
    return 2.*(predt - y)

def hessian(predt: np.ndarray, dtrain: np.ndarray) -> np.ndarray:
    y = dtrain.get_label()
    return 2.*np.ones_like(predt)

def my_quadratic(predt: np.ndarray, dtrain: np.ndarray)  -> np.ndarray:
    
    grad = gradient(predt, dtrain)
    hess = hessian(predt, dtrain)
    return grad, hess

In [76]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

weight = torch.empty(len(X_train)).uniform_()
weight = weight/weight.mean()

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest  = xgb.DMatrix(X_test)

regressor = xgb.train({'tree_method': 'hist', 'seed': 1994},
                      dtrain=dtrain,
                      num_boost_round=100,
                      obj=squared_log)

y_pred = regressor.predict(dtest)

mean_squared_error(y_test, y_pred)

109.77569745300839

In [82]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

weight = torch.empty(len(X_train)).uniform_()
weight = weight/weight.mean()

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest  = xgb.DMatrix(X_test)

regressor = xgb.train({'tree_method': 'hist', 'seed': 1994},
                      dtrain=dtrain,
                      num_boost_round=100,
                      obj=my_quadratic)

y_pred = regressor.predict(dtest)

mean_squared_error(y_test, y_pred)

6.076948776711798